# Cost Allocation from Constraint Matrix

### Import packages

In [1]:
import pypsa
import pandas as pd
from helpers import get_linear_system, noisy_lopf
import networks

## Load example network

### One Bus, One Snapshot, Two Generators without invesment

$G_1 < d < G_1 + G_2$

Note, for such a system the total system cost $TC$ are less then the nodal payments as soon as one generator is at its limit: 

$\lambda = o_s + \bar{\mu_s} \;\;\;\;\; \forall s$

$d \, \lambda = d \, (o_s + \bar{\mu_s} ) \ge \sum_s g_s \, o_s = TC$

In [2]:
n = networks.n1_t9_g2_w()
noisy_lopf(n)

s = get_linear_system(n)
A_, A_inv, c, x, d, m, r = (s[k].round(1) for k in ['A_', 'A_inv', 'c', 'x', 'd', 'm', 'r'])

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.13s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Using license file /opt/gurobi900/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /tmp/pypsa-problem-tt7vu9j7.lp
Reading time = 0.00 seconds
obj: 49 rows, 21 columns, 94 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 49 rows, 21 columns and 94 nonzeros
Model fingerprint: 0xcbbd7898
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 9e+01]
Presolve removed 22 rows and 10 columns
Presolve time: 0.01s
Presolved: 27 rows, 11 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0937103e+03   5.917013e+01   0.000000e+00      0s
      11    4.7378964e+03   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective  4.737896438e+03


INFO:pypsa.linopf:Optimization successful. Objective value: 4.74e+03


In [3]:
print(n.objective)
print(n.objective_constant)

4737.896438079306
0.0


In [4]:
n.loads_t.p_set

,0
0,50.0
1,49.0
2,48.0
3,47.0
4,46.0
5,90.0
6,46.5
7,47.5
8,48.5


In [5]:
n.generators_t.p.round(2)

,Gen0,Gen1
0,47.0,3.0
1,47.0,2.0
2,47.0,1.0
3,47.0,0.0
4,46.0,0.0
5,47.0,43.0
6,46.5,0.0
7,47.0,0.5
8,47.0,1.5


In [6]:
n.generators[['p_nom', 'p_nom_extendable', 'p_nom_opt', 'marginal_cost', 'capital_cost']].round(1)

,p_nom,p_nom_extendable,p_nom_opt,marginal_cost,capital_cost
Gen0,0.0,True,47.0,2.0,50.0
Gen1,0.0,True,43.0,5.0,30.0


# Understand the Cost Allocation 


Summerizing equations:

$\sum_i x_i \, A'_{i,j} = d_j \hspace{10pt} \leftrightarrow \hspace{10pt} x_i = \sum_j {A'}_{i,j}^{-1} \, d_j$

$\sum_j A'_{i,j} \, \mu_j = c_i \hspace{10pt} \leftrightarrow \hspace{10pt} \mu_j = \sum_i c_i \, {A'}_{i,j}^{-1}$

The total cost can be represented through all of the following expressions

$TC = \sum_i c_i \, x_i = \sum_{i,j}  c_i \, {A'}^{-1}_{ij} d_j = \sum_{i,j} x_i \, A'_{i,j} \, \mu_j = \sum_j \mu_j \, d_j $

In [7]:
assert all(A_.T @ x == d)

assert all(A_ @ m == c)

assert round(n.objective, 0) ==  A_inv @ d @ c == A_ @ m @ x

The basis of the cost allocation is ${A'}^{-1}$. It connects binding constraint to the variables

$x_i = \sum_j {A'}^{-1}_{i,j} \, d_j$

If ${A'}^{-1}_{i,j}$ is positive, $d_j$ has a positive effect on $x_i$. If negative, $d_j$ pushes $x_i$ down. 

In [8]:
A_inv * d

component                            Generator                                \
name                                  mu_upper                                 
component_i                               Gen0                     Gen1 Gen0   
snapshot                                     0    1    2    3    5    5    7   
component name  component_i snapshot                                           
Generator p     Gen0        0             -0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        0              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        1              0.0 -0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        1              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        2              0.0  0.0 -0.0  0.0 -0.0 -0.0  0.0   
                Gen1        2              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        3              0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        3              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        4              0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        4              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        5              0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        5              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        6              0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        6              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        7              0.0  0.0  0.0  0.0 -0.0 -0.0 -0.0   
                Gen1        7              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
                Gen0        8              0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        8              0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
          p_nom Gen0        static         0.0  0.0  0.0  0.0 -0.0 -0.0  0.0   
                Gen1        static         0.0  0.0  0.0 -0.0  0.0  0.0  0.0   

component                                                               Bus  \
name                                      mu_lower           marginal_price   
component_i                                   Gen1                     Bus1   
snapshot                                8        3    4    6              0   
component name  component_i snapshot                                          
Generator p     Gen0        0         0.0      0.0  0.0  0.0            0.0   
                Gen1        0         0.0     -0.0  0.0  0.0           50.0   
                Gen0        1         0.0      0.0  0.0  0.0            0.0   
                Gen1        1         0.0     -0.0  0.0  0.0            0.0   
                Gen0        2         0.0      0.0  0.0  0.0            0.0   
                Gen1        2         0.0     -0.0  0.0  0.0            0.0   
                Gen0        3         0.0      0.0  0.0  0.0            0.0   
                Gen1        3         0.0     -0.0  0.0  0.0            0.0   
                Gen0        4         0.0      0.0  0.0  0.0            0.0   
                Gen1        4         0.0     -0.0 -0.0  0.0            0.0   
                Gen0        5         0.0      0.0  0.0  0.0            0.0   
                Gen1        5         0.0     -0.0  0.0  0.0            0.0   
                Gen0        6         0.0      0.0  0.0  0.0            0.0   
                Gen1        6         0.0     -0.0  0.0 -0.0            0.0   
                Gen0        7         0.0      0.0  0.0  0.0            0.0   
                Gen1        7         0.0     -0.0  0.0  0.0            0.0   
                Gen0        8        -0.0      0.0  0.0  0.0            0.0   
                Gen1        8         0.0     -0.0  0.0  0.0            0.0   
          p_nom Gen0        static    0.0      0.0  0.0  0.0            0.0   
                Gen1        static    0.0     -0.0  0.0  0.0            0.0   

component              

The same counts for the cost of the variables $C_{i,j}$ defined as  

$C_{ij} = {A'}^{-1}_{ij} \, c_i \, d_j \hspace{10pt} \forall j \in \{j \, | \, d_j > 0\}$

If $C_{i,j}$ is positive, constraint $j$ pushes expences for variables $i$ up, if negative it lowers them. 

In [9]:
C = (r).mul(c, 0)
assert round((r.T @ c).sum(), 0) == round(n.objective, 0)
C

component                                       Bus                        \
name                                 marginal_price                         
component_i                                    Bus1                         
snapshot                                          0      1      2       3   
component name  component_i snapshot                                        
Generator p     Gen0        0                   0.0    0.0    0.0    94.0   
                Gen1        0                 250.0    0.0    0.0  -235.0   
                Gen0        1                   0.0    0.0    0.0    94.0   
                Gen1        1                   0.0  245.0    0.0  -235.0   
                Gen0        2                   0.0    0.0    0.0    94.0   
                Gen1        2                   0.0    0.0  240.0  -235.0   
                Gen0        3                   0.0    0.0    0.0    94.0   
                Gen1        3                   0.0    0.0    0.0    -0.0   
                Gen0        4                   0.0    0.0    0.0     0.0   
                Gen1        4                   0.0    0.0    0.0    -0.0   
                Gen0        5                   0.0    0.0    0.0    94.0   
                Gen1        5                   0.0    0.0    0.0  -235.0   
                Gen0        6                   0.0    0.0    0.0     0.0   
                Gen1        6                   0.0    0.0    0.0    -0.0   
                Gen0        7                   0.0    0.0    0.0    94.0   
                Gen1        7                   0.0    0.0    0.0  -235.0   
                Gen0        8                   0.0    0.0    0.0    94.0   
                Gen1        8                   0.0    0.0    0.0  -235.0   
          p_nom Gen0        static              0.0    0.0    0.0  2350.0   
                Gen1        static              0.0    0.0    0.0 -1410.0   

component                                                               
name                                                                    
component_i                                                             
snapshot                                 4       5     6      7      8  
component name  component_i snapshot                                    
Generator p     Gen0        0          0.0    -0.0   0.0    0.0    0.0  
                Gen1        0          0.0     0.0   0.0    0.0    0.0  
                Gen0        1          0.0    -0.0   0.0    0.0    0.0  
                Gen1        1          0.0     0.0   0.0    0.0    0.0  
                Gen0        2          0.0    -0.0   0.0    0.0    0.0  
                Gen1        2          0.0     0.0   0.0    0.0    0.0  
                Gen0        3          0.0    -0.0   0.0    0.0    0.0  
                Gen1        3          0.0     0.0   0.0    0.0    0.0  
                Gen0        4         92.0    -0.0   0.0    0.0    0.0  
                Gen1        4          0.0     0.0   0.0    0.0    0.0  
                Gen0        5          0.0    -0.0   0.0    0.0    0.0  
                Gen1        5          0.0   450.0   0.0    0.0    0.0  
                Gen0        6          0.0    -0.0  93.0    0.0    0.0  
                Gen1        6          0.0     0.0   0.0    0.0    0.0  
                Gen0        7          0.0    -0.0   0.0    0.0    0.0  
                Gen1        7          0.0     0.0   0.0  237.5    0.0  
                Gen0        8          0.0    -0.0   0.0    0.0    0.0  
                Gen1        8          0.0     0.0   0.0    0.0  242.5  
          p_nom Gen0        static     0.0    -0.0   0.0    0.0    0.0  
                Gen1        static     0.0  2700.0   0.0    0.0    0.0

**Here seems to be the problem:** 

$C_{i,j}$ has values below zero, see first two rows and have a look at column 3. Figuratively, this means that consumers at Bus1 and snapshot 3 receive money from Gen1, and redistribute it to Gen0. Since this happens also for other productions cost, it looses the uniqueness of the peer-to-peer payment. More concrete: We cannot say what amount consumers at Bus1 and snapshot 0 effectively pay to Gen0 at snapshot 0.

In [10]:
C.sum().to_frame('Constraint induced cost')

Constraint induced cost
component name           component_i snapshot                         
Bus       marginal_price Bus1        0                           250.0
                                     1                           245.0
                                     2                           240.0
                                     3                           188.0
                                     4                            92.0
                                     5                          3150.0
                                     6                            93.0
                                     7                           237.5
                                     8                           242.5

In [11]:
C.sum(1).to_frame('Cost per variable')

Cost per variable
component name  component_i snapshot                   
Generator p     Gen0        0                      94.0
                Gen1        0                      15.0
                Gen0        1                      94.0
                Gen1        1                      10.0
                Gen0        2                      94.0
                Gen1        2                       5.0
                Gen0        3                      94.0
                Gen1        3                       0.0
                Gen0        4                      92.0
                Gen1        4                       0.0
                Gen0        5                      94.0
                Gen1        5                     215.0
                Gen0        6                      93.0
                Gen1        6                       0.0
                Gen0        7                      94.0
                Gen1        7                       2.5
                Gen0        8                      94.0
                Gen1        8                       7.5
          p_nom Gen0        static               2350.0
                Gen1        static               1290.0

### Other direction

The matrix $A'_{i,j}$ connects to the variables their binding constraints

$\sum_i x_i \, A'_{i,j} = d_j$


In [12]:
A_.mul(x, 0)

component                            Generator                                \
name                                  mu_upper                                 
component_i                               Gen0                          Gen1   
snapshot                                     0     1     2     3     5     5   
component name  component_i snapshot                                           
Generator p     Gen0        0            -47.0   0.0   0.0   0.0   0.0   0.0   
                Gen1        0              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        1              0.0 -47.0   0.0   0.0   0.0   0.0   
                Gen1        1              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        2              0.0   0.0 -47.0   0.0   0.0   0.0   
                Gen1        2              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        3              0.0   0.0   0.0 -47.0   0.0   0.0   
                Gen1        3              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        4              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen1        4              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        5              0.0   0.0   0.0   0.0 -47.0   0.0   
                Gen1        5              0.0   0.0   0.0   0.0   0.0 -43.0   
                Gen0        6              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen1        6              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        7              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen1        7              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen0        8              0.0   0.0   0.0   0.0   0.0   0.0   
                Gen1        8              0.0   0.0   0.0   0.0   0.0   0.0   
          p_nom Gen0        static        47.0  47.0  47.0  47.0  47.0   0.0   
                Gen1        static         0.0   0.0   0.0   0.0   0.0  43.0   

component                                                            \
name                                             mu_lower             
component_i                           Gen0           Gen1             
snapshot                                 7     8        3    4    6   
component name  component_i snapshot                                  
Generator p     Gen0        0          0.0   0.0      0.0  0.0  0.0   
                Gen1        0          0.0   0.0      0.0  0.0  0.0   
                Gen0        1          0.0   0.0      0.0  0.0  0.0   
                Gen1        1          0.0   0.0      0.0  0.0  0.0   
                Gen0        2          0.0   0.0      0.0  0.0  0.0   
                Gen1        2          0.0   0.0      0.0  0.0  0.0   
                Gen0        3          0.0   0.0      0.0  0.0  0.0   
                Gen1        3          0.0   0.0     -0.0  0.0  0.0   
                Gen0        4          0.0   0.0      0.0  0.0  0.0   
                Gen1        4          0.0   0.0      0.0 -0.0  0.0   
                Gen0        5          0.0   0.0      0.0  0.0  0.0   
                Gen1        5          0.0   0.0      0.0  0.0  0.0   
                Gen0        6          0.0   0.0      0.0  0.0  0.0   
                Gen1        6          0.0   0.0      0.0  0.0 -0.0   
                Gen0        7        -47.0   0.0      0.0  0.0  0.0   
                Gen1        7          0.0   0.0      0.0  0.0  0.0   
                Gen0        8          0.0 -47.0      0.0  0.0  0.0   
                Gen1        8          0.0   0.0      0.0  0.0  0.0   
          p_nom Gen0        static    47.0  47.0      0.0  0.0  0.0   
                Gen1        static     0.0   0.0      0.0  0.0  0.0   

component                                       Bus                          \
name                                 marginal_price                           
component_i                                    Bus1              

In [13]:
D = (A_ * m).mul(x, 0) 
assert round(n.objective, 0) ==  D.sum().sum()
D

component                            Generator                              \
name                                  mu_upper                               
component_i                               Gen0                               
snapshot                                     0      1      2     3       5   
component name  component_i snapshot                                         
Generator p     Gen0        0           -141.0    0.0    0.0   0.0     0.0   
                Gen1        0              0.0    0.0    0.0   0.0     0.0   
                Gen0        1              0.0 -141.0    0.0   0.0     0.0   
                Gen1        1              0.0    0.0    0.0   0.0     0.0   
                Gen0        2              0.0    0.0 -141.0   0.0     0.0   
                Gen1        2              0.0    0.0    0.0   0.0     0.0   
                Gen0        3              0.0    0.0    0.0 -94.0     0.0   
                Gen1        3              0.0    0.0    0.0   0.0     0.0   
                Gen0        4              0.0    0.0    0.0   0.0     0.0   
                Gen1        4              0.0    0.0    0.0   0.0     0.0   
                Gen0        5              0.0    0.0    0.0   0.0 -1551.0   
                Gen1        5              0.0    0.0    0.0   0.0     0.0   
                Gen0        6              0.0    0.0    0.0   0.0     0.0   
                Gen1        6              0.0    0.0    0.0   0.0     0.0   
                Gen0        7              0.0    0.0    0.0   0.0     0.0   
                Gen1        7              0.0    0.0    0.0   0.0     0.0   
                Gen0        8              0.0    0.0    0.0   0.0     0.0   
                Gen1        8              0.0    0.0    0.0   0.0     0.0   
          p_nom Gen0        static       141.0  141.0  141.0  94.0  1551.0   
                Gen1        static         0.0    0.0    0.0   0.0     0.0   

component                                                                      \
name                                                       mu_lower             
component_i                             Gen1   Gen0            Gen1             
snapshot                                   5      7      8        3    4    6   
component name  component_i snapshot                                            
Generator p     Gen0        0            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        0            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen0        1            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        1            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen0        2            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        2            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen0        3            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        3            0.0    0.0    0.0      0.0 -0.0 -0.0   
                Gen0        4            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        4            0.0    0.0    0.0     -0.0  0.0 -0.0   
                Gen0        5            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        5        -1290.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen0        6            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen1        6            0.0    0.0    0.0     -0.0 -0.0  0.0   
                Gen0        7            0.0 -141.0    0.0     -0.0 -0.0 -0.0   
                Gen1        7            0.0    0.0    0.0     -0.0 -0.0 -0.0   
                Gen0        8            0.0    0.0 -141.0     -0.0 -0.0 -0.0   
                Gen1        8            0.0    0.0    0.0     -0.0 -0.0 -0.0   
          p_nom Gen0        static       0.0  141.0  141.0     -0.0 -0.0 -0.0   
                Gen1        static    1290.0    0.0    0.0     -0.0 -0.0 -0.0   

component              

## How to deal with other non-zero rhs?

We want $TC$ to be totally covered by the consumers $\sum_{n,t} \lambda_{n,t} \, d_{n,t}$. However $d_{n,t}$ is only a subset of $d_j$ and $TC$ might well be greater the $\sum_{n,t} \lambda_{n,t} \, d_{n,t}$. So we aim at deriving a alternated LMP $\lambda^*$ which account for all other binding contraints with a non-zero rhs, $d_j > 0$.

Let KCL denote the set of constraints which define all nodal balances. Then 


$TC = \sum_{j \in KCL} \mu_j \, d_j + \sum_{j \notin KCL} \mu_j \, d_j = \sum_{j \in KCL} \mu^*_j \, d_j$

We define the alternate cost coefficients $c_i^*$ which fulfill

$c_i = \sum_{j \in KCL} A'_{i,j} \, \mu_j + \sum_{j \notin KCL} A'_{i,j} \, \mu_j = \sum_{j \in KCL} A'_{i,j} \, \mu^*_j $

$c^*_i = c_i - \sum_{j \notin KCL} A'_{i,j} = \sum_{j \in KCL} A'_{i,j} \, \mu_j$ 

In [14]:
c

component  name   component_i  snapshot
Generator  p      Gen0         0            2.0
                  Gen1         0            5.0
                  Gen0         1            2.0
                  Gen1         1            5.0
                  Gen0         2            2.0
                  Gen1         2            5.0
                  Gen0         3            2.0
                  Gen1         3            5.0
                  Gen0         4            2.0
                  Gen1         4            5.0
                  Gen0         5            2.0
                  Gen1         5            5.0
                  Gen0         6            2.0
                  Gen1         6            5.0
                  Gen0         7            2.0
                  Gen1         7            5.0
                  Gen0         8            2.0
                  Gen1         8            5.0
           p_nom  Gen0         static      50.0
                  Gen1         static      30.0


In [15]:
kcl_i = d.loc[['Bus'], ['marginal_price'], :, :].index
c_new = A_[kcl_i] @ m[kcl_i] 
c_new

component  name   component_i  snapshot
Generator  p      Gen0         0            5.0
                  Gen1         0            5.0
                  Gen0         1            5.0
                  Gen1         1            5.0
                  Gen0         2            5.0
                  Gen1         2            5.0
                  Gen0         3            4.0
                  Gen1         3            4.0
                  Gen0         4            2.0
                  Gen1         4            2.0
                  Gen0         5           35.0
                  Gen1         5           35.0
                  Gen0         6            2.0
                  Gen1         6            2.0
                  Gen0         7            5.0
                  Gen1         7            5.0
                  Gen0         8            5.0
                  Gen1         8            5.0
           p_nom  Gen0         static       0.0
                  Gen1         static       0.0


In [16]:
A_inv[kcl_i].mul(c_new, 0) * d[kcl_i]

component                                       Bus                        \
name                                 marginal_price                         
component_i                                    Bus1                         
snapshot                                          0      1      2       3   
component name  component_i snapshot                                        
Generator p     Gen0        0                   0.0    0.0    0.0   235.0   
                Gen1        0                 250.0    0.0    0.0  -235.0   
                Gen0        1                   0.0    0.0    0.0   235.0   
                Gen1        1                   0.0  245.0    0.0  -235.0   
                Gen0        2                   0.0    0.0    0.0   235.0   
                Gen1        2                   0.0    0.0  240.0  -235.0   
                Gen0        3                   0.0    0.0    0.0   188.0   
                Gen1        3                   0.0    0.0    0.0    -0.0   
                Gen0        4                   0.0    0.0    0.0     0.0   
                Gen1        4                   0.0    0.0    0.0    -0.0   
                Gen0        5                   0.0    0.0    0.0  1645.0   
                Gen1        5                   0.0    0.0    0.0 -1645.0   
                Gen0        6                   0.0    0.0    0.0     0.0   
                Gen1        6                   0.0    0.0    0.0    -0.0   
                Gen0        7                   0.0    0.0    0.0   235.0   
                Gen1        7                   0.0    0.0    0.0  -235.0   
                Gen0        8                   0.0    0.0    0.0   235.0   
                Gen1        8                   0.0    0.0    0.0  -235.0   
          p_nom Gen0        static              0.0    0.0    0.0     0.0   
                Gen1        static              0.0    0.0    0.0    -0.0   

component                                                               
name                                                                    
component_i                                                             
snapshot                                 4       5     6      7      8  
component name  component_i snapshot                                    
Generator p     Gen0        0          0.0    -0.0   0.0    0.0    0.0  
                Gen1        0          0.0     0.0   0.0    0.0    0.0  
                Gen0        1          0.0    -0.0   0.0    0.0    0.0  
                Gen1        1          0.0     0.0   0.0    0.0    0.0  
                Gen0        2          0.0    -0.0   0.0    0.0    0.0  
                Gen1        2          0.0     0.0   0.0    0.0    0.0  
                Gen0        3          0.0    -0.0   0.0    0.0    0.0  
                Gen1        3          0.0     0.0   0.0    0.0    0.0  
                Gen0        4         92.0    -0.0   0.0    0.0    0.0  
                Gen1        4          0.0     0.0   0.0    0.0    0.0  
                Gen0        5          0.0    -0.0   0.0    0.0    0.0  
                Gen1        5          0.0  3150.0   0.0    0.0    0.0  
                Gen0        6          0.0    -0.0  93.0    0.0    0.0  
                Gen1        6          0.0     0.0   0.0    0.0    0.0  
                Gen0        7          0.0    -0.0   0.0    0.0    0.0  
                Gen1        7          0.0     0.0   0.0  237.5    0.0  
                Gen0        8          0.0    -0.0   0.0    0.0    0.0  
                Gen1        8          0.0     0.0   0.0    0.0  242.5  
          p_nom Gen0        static     0.0    -0.0   0.0    0.0    0.0  
                Gen1        static     0.0     0.0   0.0    0.0    0.0